In [ ]:
!pip install --quiet pymupdf pillow pytesseract reportlab groq tqdm langchain_groq

In [11]:
# Install dependencies
!pip install fpdf2 pdf2image pytesseract pillow --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 10.5 MB/s eta 0:00:00


In [14]:
!apt-get install -y fonts-dejavu-core > /dev/null

In [8]:
from pathlib import Path

# INPUT / OUTPUT
INPUT_PDF = Path("input_scanned.pdf")
OUT_DIR    = Path("outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

LANGS = "eng"
PDF_OCR_OUT = OUT_DIR / "searchable_ocrmypdf.pdf"

In [15]:
# =====================================
# STEP 1: SIMPLE OCR APPROACH (Tesseract)
# =====================================

from pathlib import Path
from fpdf import FPDF
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import os

# ==== INPUT / OUTPUT ====
INPUT_PDF = Path("input_scanned.pdf")          # your input file
OUT_DIR = Path("outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

LANGS = "eng"
PDF_OCR_OUT = OUT_DIR / "searchable_ocrmypdf.pdf"

# ==== Convert PDF pages to images ====
pages = convert_from_path(INPUT_PDF, dpi=300)
temp_dir = OUT_DIR / "temp_images"
temp_dir.mkdir(exist_ok=True)

# ==== OCR Each Page ====
ocr_texts = []
for i, page in enumerate(pages):
    img_path = temp_dir / f"page_{i+1}.png"
    page.save(img_path, "PNG")
    text = pytesseract.image_to_string(Image.open(img_path), lang=LANGS)
    ocr_texts.append(text)
    print(f"OCR completed for page {i+1}/{len(pages)}")

# ==== Create a new PDF with Unicode support ====
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_font("DejaVu", "", fname="/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", uni=True)

for i, text in enumerate(ocr_texts):
    pdf.add_page()
    pdf.set_font("DejaVu", size=12)
    pdf.multi_cell(0, 10, text)
    pdf.ln()

pdf.output(str(PDF_OCR_OUT))
print(f"\n✅ OCR text PDF saved successfully at: {PDF_OCR_OUT.resolve()}")


OCR completed for page 1/8
OCR completed for page 2/8
OCR completed for page 3/8
OCR completed for page 4/8
OCR completed for page 5/8
OCR completed for page 6/8
OCR completed for page 7/8
OCR completed for page 8/8

✅ OCR text PDF saved successfully at: /content/outputs/searchable_ocrmypdf.pdf


In [ ]:
# Install dependencies
!pip install pdf2image langchain langchain-groq pillow fpdf pytesseract

In [ ]:
# ===========================================
# STEP 2: GROQ VISION MODEL APPROACH (Direct API)
# ===========================================

from pathlib import Path
from groq import Groq
from pdf2image import convert_from_path
from fpdf import FPDF
import os, base64, glob

# ==== INPUT / OUTPUT ====
INPUT_PDF = Path("input_scanned.pdf")
OUT_DIR = Path("outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)
PDF_GROQ_OUT = OUT_DIR / "pdf_groq.pdf"

# ==== Setup your Groq API Key ====
GROQ_API_KEY = input("🔑 Enter your GROQ API Key: ").strip()
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

# ==== Locate Unicode-safe font ====
font_candidates = glob.glob("/usr/share/fonts/**/DejaVuSans.ttf", recursive=True)
FONT_PATH = font_candidates[0] if font_candidates else None
if not FONT_PATH:
    raise RuntimeError("❌ DejaVuSans.ttf not found. Try installing it manually.")
print(f"✅ Using font: {FONT_PATH}")

# ==== Convert PDF pages to images ====
pages = convert_from_path(INPUT_PDF, dpi=200)
image_dir = OUT_DIR / "groq_images"
image_dir.mkdir(exist_ok=True)

image_paths = []
for i, page in enumerate(pages):
    img_path = image_dir / f"page_{i+1}.png"
    page.save(img_path, "PNG")
    image_paths.append(img_path)

print(f"✅ Converted {len(image_paths)} pages to images")

# ==== Initialize Groq Client ====
client = Groq(api_key=os.environ["GROQ_API_KEY"])

# ==== Helper: Encode image ====
def encode_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")

# ==== Process each page ====
structured_texts = []

for i, img_path in enumerate(image_paths):
    print(f"🔍 Processing page {i+1}/{len(image_paths)} with Groq Vision...")

    base64_img = encode_image(img_path)
    prompt_text = """You are a vision model specialized in document OCR and layout understanding.

Extract *all* readable content (paragraphs, tables, headers) from this scanned page image.
Output the result in **Markdown format**, using:
- #, ## for headers
- Plain text for paragraphs
- Markdown tables for any visible tables
Preserve spacing and order of content.
"""

    chat_completion = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_text},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_img}",
                        },
                    },
                ],
            }
        ],
    )

    structured_text = chat_completion.choices[0].message.content
    structured_texts.append(structured_text)
    print(f"✅ Page {i+1} processed.")

# ==== Create final PDF ====
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_font("DejaVu", "", FONT_PATH, uni=True)

for i, text in enumerate(structured_texts):
    pdf.add_page()
    pdf.set_font("DejaVu", size=11)
    pdf.multi_cell(0, 8, text)
    pdf.ln()

pdf.output(str(PDF_GROQ_OUT))
print(f"\n✅ Groq Vision extracted PDF saved at: {PDF_GROQ_OUT.resolve()}")
